In [4]:
!pip install --upgrade mxnet
!pip install autogluon

  Using cached autogluon-0.8.2-py3-none-any.whl (9.7 kB)
  Using cached autogluon.core-0.8.2-py3-none-any.whl (224 kB)
  Using cached autogluon.features-0.8.2-py3-none-any.whl (62 kB)
  Using cached autogluon.tabular-0.8.2-py3-none-any.whl (285 kB)
  Using cached autogluon.multimodal-0.8.2-py3-none-any.whl (372 kB)
  Using cached autogluon.timeseries-0.8.2-py3-none-any.whl (116 kB)
  Using cached boto3-1.28.8-py3-none-any.whl (135 kB)
  Using cached autogluon.common-0.8.2-py3-none-any.whl (61 kB)
  Using cached seqeval-1.2.2-py3-none-any.whl
  Using cached evaluate-0.3.0-py3-none-any.whl (72 kB)
  Using cached accelerate-0.16.0-py3-none-any.whl (199 kB)
  Using cached timm-0.9.2-py3-none-any.whl (2.2 MB)
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Using cached torchvision-0.14.1-cp310-cp310-manylinux1_x86_64.whl (24.2 MB)
  Using cached pytorch_lightning-1.9.5-py3-none-any.whl (829 kB)
  Using cached torchmetrics-0.11.4-py3-none-any.whl (519 kB)
  Using c

In [5]:
import autogluon as ag

In [6]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd


In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv', index_col=0)

df.head()
df.index.name = 'date'
df.index = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
df.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
df = df[24:]
df.describe()



,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir
count,41757.000000,43800.000000,43800.000000,43800.000000,43800.000000,43800.000000,43800.000000
mean,98.613215,1.828516,12.459041,1016.447306,23.894307,0.052763,0.195023
std,92.050387,14.429326,12.193384,10.271411,50.022729,0.760582,1.416247
min,0.000000,-40.000000,-19.000000,991.000000,0.450000,0.000000,0.000000
25%,29.000000,-10.000000,2.000000,1008.000000,1.790000,0.000000,0.000000
50%,72.000000,2.000000,14.000000,1016.000000,5.370000,0.000000,0.000000
75%,137.000000,15.000000,23.000000,1025.000000,21.910000,0.000000,0.000000
max,994.000000,28.000000,42.000000,1046.000000,585.600000,27.000000,36.000000


In [8]:
d = df['pm2.5']
s_fwd = d.interpolate(method="polynomial", order=3, limit_direction='forward')
s_bkd = d.interpolate(method="polynomial", order=3, limit_direction='backward')
print(s_fwd.count())
print(s_bkd.count())
type(s_bkd)
tmp_df = pd.DataFrame()
tmp_df['fwd'] = s_fwd
tmp_df['bkd'] = s_bkd
tmp_df['mean'] = tmp_df.mean(axis=1)
tmp_df['mean'][tmp_df['mean'] < 0] = 0

df['pm2.5'] = tmp_df['mean']


43800
43800


In [9]:
df['time']= df.index
df_train = df[:int(df.shape[0] * 0.8)]
df_test = df[int(df.shape[0] * 0.8):]


In [10]:
train_data = TabularDataset(df_train)


In [11]:
# Set the target column and feature columns
target_col = 'pm2.5'
feature_cols = ['DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']
x = feature_cols
x.append(target_col)

In [12]:
train_data.head()

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,time
2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0,2010-01-02 00:00:00
2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0,2010-01-02 01:00:00
2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0,2010-01-02 02:00:00
2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0,2010-01-02 03:00:00
2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0,2010-01-02 04:00:00


In [13]:

# Set the target column and feature columns
target_col = 'pm2.5'
feature_cols = ['DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']
train_data = train_data[x]
# Set up the AutoML model
import time

start = time.time()
start

predictor = TabularPredictor(
    label=target_col,
    eval_metric='rmse',
    problem_type='regression',
).fit(train_data=train_data, hyperparameters='multimodal')
end = time.time()
print(end-start)



No path specified. Models will be saved in: "AutogluonModels/ag-20230720_222233/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230720_222233/"
AutoGluon Version:  0.8.2
Python Version:     3.10.6
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   81.91 GB / 115.66 GB (70.8%)
Train Data Rows:    35040
Train Data Columns: 6
Label Column: pm2.5
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12515.19 MB
	Train Data (Original)  Memory Usage: 1.68 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGe

[1000]	valid_set's rmse: 74.9534


	-74.9322	 = Validation score   (-root_mean_squared_error)
	7.66s	 = Training   runtime
	0.53s	 = Validation runtime
Fitting model: LightGBMXT ...


[1000]	valid_set's rmse: 79.074
[2000]	valid_set's rmse: 78.4147
[3000]	valid_set's rmse: 78.0625
[4000]	valid_set's rmse: 77.7938
[5000]	valid_set's rmse: 77.5952
[6000]	valid_set's rmse: 77.3888
[7000]	valid_set's rmse: 77.2737
[8000]	valid_set's rmse: 77.1274
[9000]	valid_set's rmse: 77.0089
[10000]	valid_set's rmse: 76.9079


	-76.9066	 = Validation score   (-root_mean_squared_error)
	24.22s	 = Training   runtime
	4.96s	 = Validation runtime
Fitting model: CatBoost ...
	-74.77	 = Validation score   (-root_mean_squared_error)
	106.52s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-76.2593	 = Validation score   (-root_mean_squared_error)
	1.74s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-80.4287	 = Validation score   (-root_mean_squared_error)
	109.29s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: VowpalWabbit ...
		`import vowpalwabbit` failed.
A quick tip is to install via `pip install vowpalwabbit>=9,<9.5
Fitting model: LightGBMLarge ...
	-75.4054	 = Validation score   (-root_mean_squared_error)
	2.11s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: MultiModalPredictor ...
		`import autogluon.multimodal` failed.
A quick tip is to install via `pip install autogluon.multimodal==0.8.2`.

Fitting mod

261.4454245567322


In [14]:
df_test

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,time
2014-01-01 00:00:00,24.0,-20,7.0,1014.0,NW,143.48,0,0,2014-01-01 00:00:00
2014-01-01 01:00:00,53.0,-20,7.0,1013.0,NW,147.50,0,0,2014-01-01 01:00:00
2014-01-01 02:00:00,65.0,-20,6.0,1013.0,NW,151.52,0,0,2014-01-01 02:00:00
2014-01-01 03:00:00,70.0,-20,6.0,1013.0,NW,153.31,0,0,2014-01-01 03:00:00
2014-01-01 04:00:00,79.0,-18,3.0,1012.0,cv,0.89,0,0,2014-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...
2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,NW,231.97,0,0,2014-12-31 19:00:00
2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,NW,237.78,0,0,2014-12-31 20:00:00
2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,NW,242.70,0,0,2014-12-31 21:00:00
2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,NW,246.72,0,0,2014-12-31 22:00:00


In [15]:
# Load test data and make predictions
test_data = df_test[feature_cols]
predictions = predictor.predict(test_data)


In [16]:
test_data

,DEWP,TEMP,PRES,Iws,Is,Ir
2014-01-01 00:00:00,-20,7.0,1014.0,143.48,0,0
2014-01-01 01:00:00,-20,7.0,1013.0,147.50,0,0
2014-01-01 02:00:00,-20,6.0,1013.0,151.52,0,0
2014-01-01 03:00:00,-20,6.0,1013.0,153.31,0,0
2014-01-01 04:00:00,-18,3.0,1012.0,0.89,0,0
...,...,...,...,...,...,...
2014-12-31 19:00:00,-23,-2.0,1034.0,231.97,0,0
2014-12-31 20:00:00,-22,-3.0,1034.0,237.78,0,0
2014-12-31 21:00:00,-22,-3.0,1034.0,242.70,0,0
2014-12-31 22:00:00,-22,-4.0,1034.0,246.72,0,0


In [17]:
s = predictor.evaluate(df_test)

Evaluation: root_mean_squared_error on test data: -81.11271737302965
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -81.11271737302965,
    "mean_squared_error": -6579.272919636987,
    "mean_absolute_error": -56.640086017232456,
    "r2": 0.26908807335865303,
    "pearsonr": 0.5219570617356082,
    "median_absolute_error": -40.33428192138672
}


In [18]:
predictor.leaderboard(df_test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-80.135614,-76.906600,11.101444,4.958512,24.216749,11.101444,4.958512,24.216749,1,True,2
1,NeuralNetTorch,-80.240243,-80.428685,0.085443,0.032931,109.292449,0.085443,0.032931,109.292449,1,True,5
2,LightGBMLarge,-80.508244,-75.405364,0.313023,0.101546,2.108030,0.313023,0.101546,2.108030,1,True,6
3,WeightedEnsemble_L2,-81.112717,-74.434855,1.000452,0.652912,116.623305,0.005373,0.003685,0.332208,2,True,7
4,LightGBM,-81.151835,-74.932244,0.563331,0.532101,7.660595,0.563331,0.532101,7.660595,1,True,1
5,CatBoost,-82.042920,-74.770007,0.118725,0.015579,106.522472,0.118725,0.015579,106.522472,1,True,3
6,XGBoost,-82.166913,-76.259294,0.535191,0.100160,1.741522,0.535191,0.100160,1.741522,1,True,4


In [19]:
x = predictor.leaderboard(df_test, silent=True)
type(x)

pandas.core.frame.DataFrame

In [20]:
# x.to_csv('/content/drive/MyDrive/AutoML/AutoML_AutoGluon/leaderboard.csv')

In [26]:
predictor.leaderboard(df, silent = True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-66.649797,-74.770007,0.264377,0.015579,106.522472,0.264377,0.015579,106.522472,1,True,3
1,XGBoost,-66.757951,-76.259294,1.674721,0.100160,1.741522,1.674721,0.100160,1.741522,1,True,4
2,WeightedEnsemble_L2,-66.952943,-74.434855,5.098810,0.652912,116.623305,0.003705,0.003685,0.332208,2,True,7
3,LightGBMLarge,-67.756725,-75.405364,1.010701,0.101546,2.108030,1.010701,0.101546,2.108030,1,True,6
4,LightGBM,-68.481399,-74.932244,3.820027,0.532101,7.660595,3.820027,0.532101,7.660595,1,True,1
5,LightGBMXT,-72.718676,-76.906600,65.250165,4.958512,24.216749,65.250165,4.958512,24.216749,1,True,2
6,NeuralNetTorch,-78.182061,-80.428685,0.234770,0.032931,109.292449,0.234770,0.032931,109.292449,1,True,5


In [29]:
# predictor.leaderboard(df_train, silent = True)

In [27]:
predictor.info()

{'path': 'AutogluonModels/ag-20230720_222233/',
 'label': 'pm2.5',
 'random_state': 0,
 'version': '0.8.2',
 'features': ['DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir'],
 'feature_metadata_in': <autogluon.common.features.feature_metadata.FeatureMetadata at 0x7bb1527b12a0>,
 'time_fit_preprocessing': 0.1406536102294922,
 'time_fit_training': 261.26994466781616,
 'time_fit_total': 261.41059827804565,
 'time_limit': None,
 'time_train_start': 1689891753.2397342,
 'num_rows_train': 32540,
 'num_cols_train': 6,
 'num_rows_val': 2500,
 'num_classes': None,
 'problem_type': 'regression',
 'eval_metric': 'root_mean_squared_error',
 'best_model': 'WeightedEnsemble_L2',
 'best_model_score_val': -74.43485521427398,
 'best_model_stack_level': 2,
 'num_models_trained': 7,
 'num_bag_folds': 0,
 'max_stack_level': 2,
 'max_core_stack_level': 1,
 'model_info': {'LightGBM': {'name': 'LightGBM',
   'model_type': 'LGBModel',
   'problem_type': 'regression',
   'eval_metric': 'root_mean_squared_error',
   'st

In [24]:
# import json

# with open('model_param.txt','w') as model_file:
#   # for k, v in predictor.info().items():
#     model_file.write('%s:%s\n' % (k, v))

In [28]:
s = predictor.info()

In [30]:
s['model_info']['CatBoost']

{'name': 'CatBoost',
 'model_type': 'CatBoostModel',
 'problem_type': 'regression',
 'eval_metric': 'root_mean_squared_error',
 'stopping_metric': 'root_mean_squared_error',
 'fit_time': 106.52247190475464,
 'num_classes': None,
 'quantile_levels': None,
 'predict_time': 0.015579462051391602,
 'val_score': -74.7700069861412,
 'hyperparameters': {'iterations': 10000,
  'learning_rate': 0.05,
  'random_seed': 0,
  'allow_writing_files': False,
  'eval_metric': 'RMSE'},
 'hyperparameters_fit': {'iterations': 9144},
 'hyperparameters_nondefault': [],
 'ag_args_fit': {'max_memory_usage_ratio': 1.0,
  'max_time_limit_ratio': 1.0,
  'max_time_limit': None,
  'min_time_limit': 0,
  'valid_raw_types': ['bool', 'int', 'float', 'category'],
  'valid_special_types': None,
  'ignored_type_group_special': None,
  'ignored_type_group_raw': None,
  'get_features_kwargs': None,
  'get_features_kwargs_extra': None,
  'predict_1_batch_size': None,
  'temperature_scalar': None},
 'num_features': 6,
 'feat